# パッケージインストール

In [ ]:
!pip install timm
!pip install -U onnx
!pip install -U onnxsim
!pip install -U onnxruntime

# Lite-Monoクローン

In [ ]:
!git clone https://github.com/noahzn/Lite-Mono

In [ ]:
%cd Lite-Mono

# 重みダウンロード

In [4]:
!mkdir weights

In [ ]:
!wget -O weights/lite-mono_640x192.zip 'https://surfdrive.surf.nl/files/index.php/s/CUjiK221EFLyXDY/download'
!wget -O weights/lite-mono-small_640x192.zip 'https://surfdrive.surf.nl/files/index.php/s/8cuZNH1CkNtQwxQ/download'
!wget -O weights/lite-mono-tiny_640x192.zip 'https://surfdrive.surf.nl/files/index.php/s/TFDlF3wYQy0Nhmg/download'
!wget -O weights/lite-mono-8m_640x192.zip 'https://surfdrive.surf.nl/files/index.php/s/UlkVBi1p99NFWWI/download'

!wget -O weights/lite-mono_1024x320.zip 'https://surfdrive.surf.nl/files/index.php/s/IK3VtPj6b5FkVnl/download'
!wget -O weights/lite-mono-small_1024x320.zip 'https://surfdrive.surf.nl/files/index.php/s/w8mvJMkB1dP15pu/download'
!wget -O weights/lite-mono-tiny_1024x320.zip 'https://surfdrive.surf.nl/files/index.php/s/myxcplTciOkgu5w/download'
!wget -O weights/lite-mono-8m_1024x320.zip 'https://surfdrive.surf.nl/files/index.php/s/mgonNFAvoEJmMas/download'

In [ ]:
!unzip weights/lite-mono_640x192.zip -d weights
!unzip weights/lite-mono-small_640x192.zip -d weights
!unzip weights/lite-mono-tiny_640x192.zip -d weights
!unzip weights/lite-mono-8m_640x192.zip -d weights

!unzip weights/lite-mono_1024x320.zip -d weights
!unzip weights/lite-mono-small_1024x320.zip -d weights
!unzip weights/lite-mono-tiny_1024x320.zip -d weights
!unzip weights/lite-mono-8m_1024x320.zip -d weights

# ONNX変換用にデコーダーの出力を修正

In [7]:
with open('networks/depth_decoder.py') as fp:
    file_lines = fp.read()

file_lines = file_lines.replace(
    'return self.outputs',
    'return self.outputs[("disp", 0)]',
)

with open('networks/depth_decoder.py', mode="w") as fp:
    fp.write(file_lines)

# Lite-Monoロード用関数

In [8]:
import os

import torch
from torchvision import transforms, datasets

import networks

def load_network(model='lite-mono', load_weights_folder=None, device='cuda'):
    device = torch.device('cuda')

    encoder_path = os.path.join(load_weights_folder, 'encoder.pth')
    decoder_path = os.path.join(load_weights_folder, 'depth.pth')

    encoder_dict = torch.load(encoder_path)
    decoder_dict = torch.load(decoder_path)

    feed_height = encoder_dict['height']
    feed_width = encoder_dict['width']
   
    encoder = networks.LiteMono(
        model=model,
        height=feed_height,
        width=feed_width,
    )

    model_dict = encoder.state_dict()
    encoder.load_state_dict(
        {k: v
        for k, v in encoder_dict.items() if k in model_dict})

    encoder.to(device)
    encoder.eval()

    depth_decoder = networks.DepthDecoder(encoder.num_ch_enc, scales=range(3))
    depth_model_dict = depth_decoder.state_dict()
    depth_decoder.load_state_dict(
        {k: v
        for k, v in decoder_dict.items() if k in depth_model_dict})

    depth_decoder.to(device)
    depth_decoder.eval()

    return encoder, depth_decoder

# ONNX変換用関数

In [9]:
def convert_to_onnx(
    input_shape=(640, 192), 
    output_dir='',
    encoder=None, 
    decoder=None, 
    device='cpu',
):
    os.makedirs(output_dir, exist_ok=True)

    # encoder
    input_image = torch.randn(1, 3, input_shape[1], input_shape[0]).to(device)

    input_layer_names = ['input_image']
    output_layer_names = ['features']

    torch.onnx.export(
        encoder, 
        input_image,
        file_name + '/encoder.onnx', 
        verbose=True,
        input_names=input_layer_names,
        output_names=output_layer_names,
        do_constant_folding=False,
        opset_version=13,
    )
    
    # decoder
    encoder_results = encoder(input_image)

    features = []
    features.append(torch.randn(*list(encoder_results[0].shape)).to(device))
    features.append(torch.randn(*list(encoder_results[1].shape)).to(device))
    features.append(torch.randn(*list(encoder_results[2].shape)).to(device))

    input_layer_names = ['features_1', 'features_2', 'features_3']
    output_layer_names = ['depth']

    torch.onnx.export(
        decoder, 
        features,
        file_name + '/decoder.onnx', 
        verbose=True,
        input_names=input_layer_names,
        output_names=output_layer_names,
        do_constant_folding=False,
        opset_version=13,
    )

    !onnxsim $file_name'/encoder.onnx' $file_name'/encoder.onnx'
    !onnxsim $file_name'/decoder.onnx' $file_name'/decoder.onnx'

# ONNX変換

In [ ]:
file_name = 'lite-mono_640x192'
input_shape = (640, 192)
model='lite-mono'

load_weights_folder = 'weights/' + file_name
encoder, decoder = load_network(model, load_weights_folder)

convert_to_onnx(
    input_shape=input_shape, 
    output_dir=file_name, 
    encoder=encoder, 
    decoder=decoder, 
    device='cuda:0',
)

In [ ]:
file_name = 'lite-mono-small_640x192'
input_shape = (640, 192)
model='lite-mono-small'

load_weights_folder = 'weights/' + file_name
encoder, decoder = load_network(model, load_weights_folder)

convert_to_onnx(
    input_shape=input_shape, 
    output_dir=file_name, 
    encoder=encoder, 
    decoder=decoder, 
    device='cuda:0',
)

In [ ]:
file_name = 'lite-mono-tiny_640x192'
input_shape = (640, 192)
model='lite-mono-tiny'

load_weights_folder = 'weights/' + file_name
encoder, decoder = load_network(model, load_weights_folder)

convert_to_onnx(
    input_shape=input_shape, 
    output_dir=file_name, 
    encoder=encoder, 
    decoder=decoder, 
    device='cuda:0',
)

In [ ]:
file_name = 'lite-mono-8m_640x192'
input_shape = (640, 192)
model='lite-mono-8m'

load_weights_folder = 'weights/' + file_name
encoder, decoder = load_network(model, load_weights_folder)

convert_to_onnx(
    input_shape=input_shape, 
    output_dir=file_name, 
    encoder=encoder, 
    decoder=decoder, 
    device='cuda:0',
)

In [ ]:
file_name = 'lite-mono_1024x320'
input_shape = (1024, 320)
model='lite-mono'

load_weights_folder = 'weights/' + file_name
encoder, decoder = load_network(model, load_weights_folder)

convert_to_onnx(
    input_shape=input_shape, 
    output_dir=file_name, 
    encoder=encoder, 
    decoder=decoder, 
    device='cuda:0',
)

In [ ]:
file_name = 'lite-mono-small_1024x320'
input_shape = (1024, 320)
model='lite-mono-small'

load_weights_folder = 'weights/' + file_name
encoder, decoder = load_network(model, load_weights_folder)

convert_to_onnx(
    input_shape=input_shape, 
    output_dir=file_name, 
    encoder=encoder, 
    decoder=decoder, 
    device='cuda:0',
)

In [ ]:
file_name = 'lite-mono-tiny_1024x320'
input_shape = (1024, 320)
model='lite-mono-tiny'

load_weights_folder = 'weights/' + file_name
encoder, decoder = load_network(model, load_weights_folder)

convert_to_onnx(
    input_shape=input_shape, 
    output_dir=file_name, 
    encoder=encoder, 
    decoder=decoder, 
    device='cuda:0',
)

In [ ]:
file_name = 'lite-mono-8m_1024x320'
input_shape = (1024, 320)
model='lite-mono-8m'

load_weights_folder = 'weights/' + file_name
encoder, decoder = load_network(model, load_weights_folder)

convert_to_onnx(
    input_shape=input_shape, 
    output_dir=file_name, 
    encoder=encoder, 
    decoder=decoder, 
    device='cuda:0',
)

# zip

In [18]:
!mkdir onnx_weights

In [ ]:
!zip onnx_weights/lite-mono_640x192 -r lite-mono_640x192
!zip onnx_weights/lite-mono-small_640x192 -r lite-mono-small_640x192
!zip onnx_weights/lite-mono-tiny_640x192 -r lite-mono-tiny_640x192
!zip onnx_weights/lite-mono-8m_640x192 -r lite-mono-8m_640x192
!zip onnx_weights/lite-mono_1024x320 -r lite-mono_1024x320
!zip onnx_weights/lite-mono-small_1024x320 -r lite-mono-small_1024x320
!zip onnx_weights/lite-mono-tiny_1024x320 -r lite-mono-tiny_1024x320
!zip onnx_weights/lite-mono-8m_1024x320 -r lite-mono-8m_1024x320

!zip onnx_weights -r onnx_weights